In [ ]:
import pandas as pd
import sqlite3

In [ ]:
conn = sqlite3.connect('inventory.db')

In [ ]:
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type ='table'",conn)
tables

In [ ]:
for table in tables['name']:
    print('-'*50,f'{table}','-'*50)
    print('Count of records:',pd.read_sql(f"select count(*) as count from {table}",conn)['count'].values[0])
    display(pd.read_sql(f"select *  from {table} limit 5",conn))




In [ ]:
purchases = pd.read_sql_query("select *from purchases where VendorNumber = 4466",conn)
purchases

In [ ]:
purchase_prices = pd.read_sql_query("""select * from purchase_prices where VendorNumber = 4466""",conn)
purchase_prices

In [ ]:
vendor_invoice = pd.read_sql_query("select *from vendor_invoice where VendorNumber = 4466",conn)
vendor_invoice

In [ ]:
sales = pd.read_sql_query("select *from sales where VendorNo = 4466",conn)
sales

In [ ]:
purchases.groupby(['Brand','PurchasePrice'])[['Quantity','Dollars']].sum()

In [ ]:
purchase_prices


In [ ]:
vendor_invoice['PONumber'].nunique()

In [ ]:
purchases

In [ ]:
vendor_invoice.columns

In [ ]:
sales.groupby('Brand')[['SalesDollars','SalesPrice','SalesQuantity']].sum()


In [ ]:
vendor_invoice.columns

In [ ]:
freight_summary = pd.read_sql_query("""select VendorNumber, SUM(Freight) as FreightCost
From vendor_invoice
Group BY VendorNumber""",conn)
freight_summary

In [ ]:
purchase_prices.columns

In [ ]:
purchases.columns

In [ ]:
pd.read_sql_query("""SELECT
    p.VendorNumber,
    p.VendorName,
    p.Brand,
    p.PurchasePrice,
    pp.Volume,
    pp.Price as ActualPrice,
    SUM(p.Quantity) as TotalPurchaseQuantity,
    SUM(p.Dollars) as TotalPurchaseDollars
    FROM purchases p
    JOIN purchase_prices pp
    ON p.Brand = pp.Brand
    where p.PurchasePrice > 0
    GROUP BY p.VendorNumber,p.VendorName,p.Brand
    ORDER BY  TotalPurchaseDollars""",conn)

In [ ]:
sales.columns

In [ ]:
pd.read_sql_query("""SELECT 
VendorNo,
Brand,
SUM(SalesDollars) as TotalSalesDollars,
SUM(SalesPrice) as TotalSalesPrice,
SUM(SalesQuantity) as TotalSalesQuantity,
SUM(ExciseTax) as TotalExciseTax
FROM sales
GROUP BY VendorNo,Brand
ORDER BY TotalSalesDollars""",conn)

In [ ]:
vendor_sales_summary = pd.read_sql_query("""
WITH FreightSummary AS (
    SELECT
        VendorNumber,
        SUM(Freight) AS FreightCost
    FROM vendor_invoice
    GROUP BY VendorNumber
),
PurchaseSummary AS (
    SELECT
        p.VendorNumber,
        p.VendorName,
        p.Brand,
        p.Description,
        p.PurchasePrice,
        pp.Volume,
        pp.Price as ActualPrice,
        SUM(p.Quantity) as TotalPurchaseQuantity,
        SUM(p.Dollars) as TotalPurchaseDollars
    FROM purchases p
    JOIN purchase_prices pp
        ON p.Brand = pp.Brand
    WHERE p.PurchasePrice > 0
    GROUP BY p.VendorNumber, p.VendorName, p.Brand, p.Description, p.PurchasePrice, pp.Volume
),
SalesSummary AS (
    SELECT 
        VendorNo,
        Brand,
        SUM(SalesDollars) as TotalSalesDollars,
        SUM(SalesPrice) as TotalSalesPrice,
        SUM(SalesQuantity) as TotalSalesQuantity,
        SUM(ExciseTax) as TotalExciseTax
    FROM sales
    GROUP BY VendorNo, Brand
)
SELECT
    ps.VendorNumber,
    ps.VendorName,
    ps.Brand,
    ps.Description,
    ps.PurchasePrice,
    ps.ActualPrice,
    ps.Volume,
    ps.TotalPurchaseQuantity,
    ps.TotalPurchaseDollars,
    ss.TotalSalesQuantity,
    ss.TotalSalesDollars,
    ss.TotalSalesPrice,
    ss.TotalExciseTax,
    fs.FreightCost
FROM PurchaseSummary ps
LEFT JOIN SalesSummary ss
    ON ps.VendorNumber = ss.VendorNo
    AND ps.Brand = ss.Brand
LEFT JOIN FreightSummary fs
    ON ps.VendorNumber = fs.VendorNumber
ORDER BY ps.TotalPurchaseDollars DESC
""", conn)


In [ ]:
vendor_sales_summary

In [ ]:
vendor_sales_summary.dtypes

In [ ]:
vendor_sales_summary.isnull().sum()

In [ ]:
vendor_sales_summary['VendorName'].unique()

In [ ]:
vendor_sales_summary['Volume'] = vendor_sales_summary['Volume'].astype('float64')

In [ ]:
vendor_sales_summary.fillna(0, inplace = True)

In [ ]:
vendor_sales_summary['VendorName'].str.strip()

In [ ]:
vendor_sales_summary['GrossProfit'] = vendor_sales_summary['TotalSalesDollars'] - vendor_sales_summary ['TotalPurchaseDollars']

In [ ]:
vendor_sales_summary['GrossProfit'].min()

In [ ]:
vendor_sales_summary['ProfitMargin'] =vendor_sales_summary['GrossProfit']/vendor_sales_summary['TotalSalesDollars']*100

In [ ]:
vendor_sales_summary['StockTurnover'] =vendor_sales_summary['TotalSalesQuantity']/vendor_sales_summary['TotalPurchaseQuantity']

In [ ]:
vendor_sales_summary['SalestoPurchaseRatio'] = vendor_sales_summary['TotalSalesDollars']/vendor_sales_summary['TotalPurchaseDollars']

In [ ]:
vendor_sales_summary.columns

In [ ]:
cursor =  conn.cursor()

In [ ]:
cursor.execute("""CREATE TABLE vendor_sales_summary (
    VendorNumber INT,
    VendorName VARCHAR(100),
    Brand INT,
    Description VARCHAR(100),
    PurchasePrice DECIMAL(10,2),
    Volume,
    TotalPurchaseQuantity INT,
    TotalPurrchaseDollars DECIMAL(15,2),
    TotalSalesQuantity INT,
    TotalSalesDollars DECIMAL(15,2),
    TotalSalesPrice DECIMAL(15,2),
    TotalExciseTax DECIMAL(15,2),
    FreightCost DECIMAL(15,2),
    GrossProfit DECIMAL(15,2),
    ProfitMargin DECIMAL(15,2),
    StockTurnover DECIMAL(15,2),
    SalesToPurchseRatio DECIMAL(15,2),
    PRIMARY KEY (VendorNumber, Brand)
);



""")

In [ ]:
pd.read_sql_query("select * from vendor_sales_summary",conn)

In [ ]:
vendor_sales_summary.to_sql('vendor_sales_summary',conn,if_exists = 'replace',index = False)

In [ ]:
import sqlite3
import pandas as pd
import logging
from ingestion_db import ingest_db



logging.basicConfig(
    filename="logs/get_vendor_summary.log",
    level=logging.DEBUG,
    format="%(asctime)s- %(levelname)s - %(message)s",
    filemode ="a"
)


def create_vendor_summary(conn):
    '''this function will merge the different tables to get the overall vendor summary and adding new columns in the resultant data'''
    vendor_sales_summary = pd.read_sql_query("""
        WITH FreightSummary AS (
            SELECT
                VendorNumber,
                SUM(Freight) AS FreightCost
            FROM vendor_invoice
            GROUP BY VendorNumber
        ),
        PurchaseSummary AS (
            SELECT
                p.VendorNumber,
                p.VendorName,
                p.Brand,
                p.Description,
                p.PurchasePrice,
                pp.Volume,
                pp.Price as ActualPrice,
                SUM(p.Quantity) as TotalPurchaseQuantity,
                SUM(p.Dollars) as TotalPurchaseDollars
            FROM purchases p
            JOIN purchase_prices pp
                ON p.Brand = pp.Brand
            WHERE p.PurchasePrice > 0
            GROUP BY p.VendorNumber, p.VendorName, p.Brand, p.Description, p.PurchasePrice, pp.Volume
        ),
        SalesSummary AS (
            SELECT 
                VendorNo,
                Brand,
                SUM(SalesDollars) as TotalSalesDollars,
                SUM(SalesPrice) as TotalSalesPrice,
                SUM(SalesQuantity) as TotalSalesQuantity,
                SUM(ExciseTax) as TotalExciseTax
            FROM sales
            GROUP BY VendorNo, Brand
        )
        SELECT
            ps.VendorNumber,
            ps.VendorName,
            ps.Brand,
            ps.Description,
            ps.PurchasePrice,
            ps.ActualPrice,
            ps.Volume,
            ps.TotalPurchaseQuantity,
            ps.TotalPurchaseDollars,
            ss.TotalSalesQuantity,
            ss.TotalSalesDollars,
            ss.TotalSalesPrice,
            ss.TotalExciseTax,
            fs.FreightCost
        FROM PurchaseSummary ps
        LEFT JOIN SalesSummary ss
            ON ps.VendorNumber = ss.VendorNo
            AND ps.Brand = ss.Brand
        LEFT JOIN FreightSummary fs
            ON ps.VendorNumber = fs.VendorNumber
        ORDER BY ps.TotalPurchaseDollars DESC
        """, conn)
        

    return  vendor_sales_summary


def clean_data(df):
    '''this function will clean the data'''
    # Changing datatype to float
    df['Volume'] = df['Volume'].astype('float64')

    # Filling missing values with 0
    df.fillna(0, inplace=True)

    # Removing spaces from categorical columns
    df['VendorName'] = df['VendorName'].str.strip()
    df['Description'] = df['Description'].str.strip()

    # Creating new columns for better analysis
    df['GrossProfit'] = df['TotalSalesDollars'] - df['TotalPurchaseDollars']
    df['ProfitMargin'] = (df['GrossProfit'] / df['TotalSalesDollars']) * 100
    df['StockTurnover'] = df['TotalSalesQuantity'] / df['TotalPurchaseQuantity']
    df['SalestoPurchaseRatio'] = df['TotalSalesDollars'] / df['TotalPurchaseDollars']

    return df


if __name__ == '__main__':
    #creating database connection
    conn = sqlite3.connect('inventory.db')

    logging.info('Creating Vendor Summary Table......')
    summary_df = create_vendor_summary(conn)
    logging.info(summary_df.head())

    logging.info('Cleaning Data.....')
    clean_df = clean_data(summary_df)
    logging.info(clean_df.head())

    logging.info('Ingesting data....')
    ingest_db(clean_df,'vendor_sales_summary',conn)
    logging.info('Completed')














